In [ ]:
from pathlib import Path
import numpy as np
from tqdm.auto import tqdm
import pandas
import cv2
import copy
base_dir = Path('./dump_match_pairs_kudamm')

gt_file = pandas.read_csv('/media/storage_hdd/Datasets/Kudamm_RAS2020/GroundTruth.csv')

In [ ]:
match_dict = {}
for f in tqdm(sorted(base_dir.iterdir())):
    if not f.is_file() or f.suffix != '.npz':
        continue

    im_num1 = int(f.stem[5:9])
    im_num2 = int(f.stem[15:19])
    # print(f.stem, im_num1, im_num2)
    matchfile = np.load(f)
    matches = matchfile['matches']
    confidences = matchfile['match_confidence']
    pos_matches_indices = matches>-1
    pos_matches = matches[pos_matches_indices]
    num_matches = len(pos_matches)
    weighted_matches = np.ones(num_matches) * confidences[pos_matches_indices]
    weighted_matches_sum = weighted_matches.sum()

#     if num_matches >= 4:
#         src_pts = matchfile['keypoints0'][pos_matches_indices].reshape(-1,1,2)
#         dst_pts = matchfile['keypoints1'][pos_matches].reshape(-1,1,2)

#         M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
#         matchesMask = mask.ravel()
#         # print(np.count_nonzero(matchesMask > 0))
#         ransac_matches = np.count_nonzero(matchesMask > 0)
#     else:
#         ransac_matches = num_matches
    
    # print(weighted_matches)
    if im_num1 not in match_dict:
        match_dict[im_num1] = {}
    # match_dict[im_num1][im_num2] = ransac_matches
    match_dict[im_num1][im_num2] = num_matches

In [ ]:
gt_images = gt_file.values[:, 0]
q_images =  gt_file.values[:, 1]

In [ ]:
n_values = [1, 2, 5]
best_matches_diffs = []
correct_at_n = np.zeros(len(n_values))
for idx1, im_num1 in enumerate(match_dict):
    all_matches = np.array(list(match_dict[im_num1].values()))
    # print(len(all_matches))
    match_sorted_idx = np.argsort(all_matches)[::-1]
    # print(gt_images[idx1], match_sorted_idx[:5])
    for i, n in enumerate(n_values):
        if np.any(np.in1d(q_images[match_sorted_idx][:n], gt_images[idx1])):
            correct_at_n[i:] += 1
            break
recall_at_n = correct_at_n / len(match_dict)
all_recalls = {}  # make dict for output
for i, n in enumerate(n_values):
    all_recalls[n] = recall_at_n[i]
all_recalls

In [ ]:
match_dict_ransac  # {1: 0.38738738738738737, 2: 0.5540540540540541, 5: 0.7522522522522522}